In [ ]:
import os
from Bio import ExPASy
from Bio import SwissProt
from Bio.PDB import PDBList, PDBParser
import requests
import pandas as pd
from tqdm import tqdm

In [ ]:

# Function to fetch the PDB ID with the best resolution for a given UniProt ID
def get_best_pdb_id(uniprot_id):
    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/best_structures/{uniprot_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if uniprot_id in data:
            best_structure = sorted(data[uniprot_id], key=lambda x: x['resolution'])[0]
            return best_structure['pdb_id'], best_structure['resolution']
    return None, None

In [ ]:
# Function to download a PDB file given a PDB ID
def download_pdb(pdb_id, output_dir):
    pdbl = PDBList()
    pdbl.retrieve_pdb_file(pdb_id, pdir=output_dir, file_format="pdb")

In [ ]:
# for MAC
AF_structs = pd.read_csv("/Users/talgalper/Documents/GitHub/PhD-MOC/Druggability_analysis/Fpocket/results_2024.05/fpocket_druggability.csv")
AF_low_conf_structs = pd.read_csv("/Users/talgalper/Documents/GitHub/PhD-MOC/Druggability_analysis/Fpocket/results_2024.05/af_low_conf_struct.csv")

# combine AF_structs and AF_low_conf_structs into a vector of uniprot_ids
uniprot_ids = pd.concat([AF_structs["uniprot_id"], AF_low_conf_structs["uniprot_id"]])
# get distinct uniprot_ids
uniprot_ids = uniprot_ids.drop_duplicates()
# create subset of first 10 IDs
uniprot_ids = uniprot_ids.head(10)


In [7]:
# for Ubuntu
AF_structs = pd.read_csv("/home/ubuntu/Documents/GitHub/PhD-MOC/Druggability_analysis/Fpocket/results_2024.05/fpocket_druggability.csv")
AF_low_conf_structs = pd.read_csv("/home/ubuntu/Documents/GitHub/PhD-MOC/Druggability_analysis/Fpocket/results_2024.05/af_low_conf_struct.csv")

# combine AF_structs and AF_low_conf_structs into a vector of uniprot_ids
uniprot_ids = pd.concat([AF_structs["uniprot_id"], AF_low_conf_structs["uniprot_id"]])
# get distinct uniprot_ids
uniprot_ids = uniprot_ids.drop_duplicates()
# create subset of first 10 IDs to test
#uniprot_ids = uniprot_ids.head(10)

In [8]:
# Main function to process a list of UniProt IDs
def download_structures(uniprot_ids, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for uniprot_id in tqdm(uniprot_ids, desc="Downloading structures"):
        pdb_id, resolution = get_best_pdb_id(uniprot_id)
        if pdb_id:
            print(f"Downloading PDB {pdb_id} for UniProt ID {uniprot_id} (Resolution: {resolution} Å)")
            download_pdb(pdb_id, output_dir)
        else:
            print(f"No structure found for UniProt ID {uniprot_id}")

if __name__ == "__main__":
    # Example list of UniProt IDs
    #output_dir = "/Users/talgalper/OneDrive - RMIT University/PhD/structures/PDB_query/" # MAC
    output_dir = "/home/ubuntu/Desktop/PDB_structures/" # Ubuntu

    download_structures(uniprot_ids, output_dir)

No structure found for UniProt ID A6NDP7


No structure found for UniProt ID E9PKD4


No structure found for UniProt ID M0QZD8


No structure found for UniProt ID O00116


No structure found for UniProt ID O14609


...

No structure found for UniProt ID O95992


No structure found for UniProt ID P25391


No structure found for UniProt ID P51790


No structure found for UniProt ID P58550


TypeError: '<' not supported between instances of 'NoneType' and 'float'